In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline
plt.style.use('ggplot')
plt.figure(figsize=(6,4))

UsageError: Line magic function `%` not found.


In [ ]:
# load Pandas dataframe
logins = pd.read_json('logins.json')
logins.head()

In [ ]:
logins.describe()

In [ ]:
logins.info()

#### Part 1     ‑    Exploratory    data    analysis 

Exploratory    data    analysis  The    attached       logins.json       file     contains     (simulated)     timestamps    of    user     logins     in    a    particular  geographic     location.     Aggregate     these     login     counts    based    on    15­ minute     time     intervals,    and  visualize     and    describe     the     resulting     time     series    of     login     counts     in    ways     that    best     characterize     the  underlying    patterns     of     the    demand.     Please     report/illustrate     important     features    of     the    demand,  such     as    daily     cycles.     If     there     are     data     quality     issues,    please     report     them. 


In [ ]:
type(logins['login_time'][0])

In [ ]:
time_range = pd.date_range('1970-01-01 20:00:00', '1970-04-13 19:00:00', freq='15T')
timestamp = pd.Series(np.zeros(len(time_range)), index=time_range)

In [ ]:
timestamp.head()

In [ ]:
logins = logins.sort_values('login_time') 
logins = logins.reset_index(drop=True)
logins.head()
# This fixes the data quality issue: Time was not in order

In [ ]:
p = 0 
for i in range(len(logins)):
    start = timestamp.index[p] 
    end = timestamp.index[p+1] 
    time = logins['login_time'][i]
    if start < time < end:
        timestamp[p] = timestamp[p] + 1
    else:
        while (end < time): 
            p = p + 1 
            start = timestamp.index[p]
            end = timestamp.index[p+1]
        timestamp[p] = timestamp[p] + 1

In [ ]:
plt.figure(figsize=(15,6)) 
timestamp.plot()

In [ ]:
plt.figure(figsize=(15,6))
timestamp[0:700].plot()

In [ ]:
timestamp.max()

In [ ]:
timestamp[timestamp == 73].index[0].dayofweek

Conclusion: Activity generally peaks approximately every 12 hours (early morning and early afternoon) and increases over the the course of the week, peaking on the weekend.

#### Part    2    ‑    Experiment    and    metrics    design

The    neighboring     cities    of    Gotham    and    Metropolis    have     complementary     circadian     rhythms:     on  weekdays,    Ultimate    Gotham     is    most     active    at    night,    and    Ultimate    Metropolis     is    most    active  during     the    day.    On    weekends,     there     is     reasonable    activity     in    both     cities.    However,    a     toll    bridge,     with     a     two  ­way     toll,    between     the     two     cities     causes    driver    partners     to     tend  to    be    exclusive     to    each     city.     The     Ultimate    managers    of     city    operations     for     the     two     cities    have  proposed    an    experiment     to     encourage    driver    partners     to    be    available     in    both     cities,    by  reimbursing    all     toll     costs.  1. What    would     you     choose     as     the     key    measure    of     success    of     this    experiment     in  encouraging    driver     partners     to     serve    both     cities,    and    why    would     you     choose     this    metric?  2. Describe    a    practical     experiment     you    would    design     to     compare     the    effectiveness    of     the  proposed     change     in     relation     to     the     key    measure    of     success.    Please    provide    details     on:   a. how     you    will     implement     the    experiment  b. what     statistical     test(s)     you    will     conduct     to     verify     the     significance    of     the  observation  c. how     you    would     interpret     the     results    and    provide     recommendations     to     the     city  operations     team    along    with    any     caveats. 


Solution

1. I would measure the increased number of drivers that cross over to the other city to meet demand and achieve an equilibrium in both markgets. If we discover that reimbursement of tolls had no effect on encouraging drivers to move to the city with the higher demand, then the experiment would have been a waste. 

2a. For sample size greater than 30, A one sample z-test would be appropriate. Sample is a single treatment group. In this case people who received reimbursed toll 

2b. Compute Z statistics. Find confidence interval , use the standard normal table to find if the z-stat falls in the confidence interval.

2c. Null hypothesis = There is a difference in driver patterns after reimbursing. Accept or reject the hypothesis based on result ot test

#### Part    3    ‑    Predictive     modeling

Ultimate     is     interested     in     predicting     rider     retention.    To    help    explore     this    question,    we    have  provided     a     sample    dataset     of     a     cohort    of    users    who     signed    up     for    an    Ultimate    account     in 
January    2014.    The     data    was    pulled     several    months     later;    we     consider    a    user     retained     if     they  were     “active”     (i.e.     took    a     trip)     in     the     preceding    30    days.    We     would     like     you     to    use     this    data     set     to    help    understand    what     factors    are     the    best    predictors  for     retention,    and    offer     suggestions     to     operationalize     those     insights     to    help    Ultimate.    The    data     is     in     the    attached     file     ultimate_data_challenge.json.    See    below     for    a    detailed  description     of     the    dataset.     Please     include    any     code     you    wrote     for     the    analysis    and    delete     the  dataset    when     you    have     finished     with     the     challenge.    1. Perform    any     cleaning,     exploratory    analysis,    and/or     visualizations     to    use     the    provided  data     for     this    analysis     (a     few     sentences/plots    describing     your    approach    will     suffice).    What  fraction     of     the    observed     users    were     retained?  2. Build    a    predictive     model     to     help    Ultimate    determine    whether    or    not    a    user    will    be    active  in     their    6th    month     on     the     system.    Discuss    why     you     chose     your    approach,    what  alternatives     you     considered,     and    any     concerns     you    have.    How     valid     is     your    model?  Include    any     key     indicators    of     model    performance.  3. Briefly    discuss    how     Ultimate     might     leverage     the     insights    gained     from     the    model     to  improve     its     long­  term     rider     retention     (again,    a     few     sentences    will     suffice).  

In [ ]:
with open('./ultimate_data_challenge.json') as f:
    data = json.load(f)

ultimate_df = pd.DataFrame(data)
ultimate_df.head()

In [ ]:
ultimate_df.info()

In [ ]:
ultimate_df.describe()

In [ ]:
ultimate_df[['last_trip_date','signup_date']].describe()

In [ ]:
ultimate_df['last_trip_date'][0]

In [ ]:
active_users = []
for i in range(len(ultimate_df)):
    st = pd.Timestamp(ultimate_df['signup_date'][i]) #get start date
    et = st + pd.Timedelta('181D') #6 months later
    at = et - pd.Timedelta('30D') #set point 30 days prior
    if at <= pd.Timestamp(ultimate_df['last_trip_date'][i]) <= et:
        active_users.append([1]) # now that the active condition has been met, set active user to 1
    else:
        active_users.append([0])

In [ ]:
df1 = pd.DataFrame(active_users,columns = ['active_users'])

In [ ]:

df1['active_users'].value_counts()

In [ ]:

12198/50000*100

#### Continuous Data

In [ ]:
sns.regplot(y=df1['active_users'], x=ultimate_df['avg_dist'], fit_reg = True)

From the above, the longer the average distance, the less likely one is to be active: Include in model

In [ ]:
sns.regplot(y=df1['active_users'], x=ultimate_df['surge_pct'], fit_reg = True)

No significant correlation: Exclude

In [ ]:
sns.regplot(y=ultimate_df['avg_rating_of_driver'], x=ultimate_df['avg_rating_by_driver'], fit_reg = True)

correlation is weak: investigate further

In [ ]:
sns.regplot(y=df1['active_users'], x=ultimate_df['avg_rating_by_driver'], fit_reg = True)

In [ ]:
sns.regplot(y=df1['active_users'], x=ultimate_df['avg_rating_of_driver'], fit_reg = True)

In [ ]:
sns.regplot(y=df1['active_users'], x=ultimate_df['weekday_pct'], fit_reg = True)

In [ ]:
sns.regplot(y=df1['active_users'], x=ultimate_df['trips_in_first_30_days'], fit_reg = True)

Number of trips in first 30 days: strong correlation, include in model

#### Categorical data

In [ ]:
df = pd.concat([ultimate_df, df1], axis = 1)

In [ ]:
df = df.drop(['avg_rating_of_driver', 'avg_surge','surge_pct','last_trip_date','signup_date','weekday_pct'], axis = 1)

In [ ]:
df.head()

In [ ]:
# Table of ultimate balck vs active
ub_active = pd.crosstab(index=df["active_users"], 
                            columns=df["ultimate_black_user"],
                             margins=True)   # Include row and column totals

ub_active.columns = ["Not Ultimate Black","Is Ultimate Black","rowtotal"]
ub_active.index= ["Not Active","Is Active","coltotal"]

ub_active

Ultimate Black looks like a good feature to include as more than half of active users are ultimate black users but less than 40% of overall users are ultimate black users. 

In [ ]:

# Table of phone type vs active
phone_active = pd.crosstab(index=df["active_users"], 
                            columns=df["phone"],
                             margins=True)   # Include row and column totals

phone_active.columns = ["Android","iPhone","rowtotal"]
phone_active.index= ["Not Active","Is Active","coltotal"]

phone_active

While Android users make up about 30% of the total users, they only make up 15% of active users. So the phone type does seem to be an indicator.

In [ ]:

# Table of ultimate balck vs active
city_active = pd.crosstab(index=df["active_users"], 
                            columns=df["city"],
                             margins=True)   # Include row and column totals

city_active.columns = ["Astapor","King's Landing",'Winterfell',"rowtotal"]
city_active.index= ["Not Active","Is Active","coltotal"]

city_active

Should be considered a good candidate as  Astapor and King's Landing have ratios outside of the standard active/not active rate.

#### Build predictive model

In [ ]:
#transforming categorical features into numerical to make sklearn happy
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['Astapor',"King's Landing",'Winterfell'])
LE = preprocessing.LabelEncoder()
LE.fit(['iPhone','Android'])

In [ ]:
LabelEncoder()

In [ ]:
df.info()

In [ ]:
df_drop = df.dropna()

In [ ]:
df_drop['active_users'].value_counts()

Drop missing values.

In [ ]:
df = df.dropna()
df = df.reset_index(drop = True)

In [ ]:
creation1 = le.transform(df['city']) #transform the categorical data into number labels
creation2 = LE.transform(df['phone'])

In [ ]:
df2 = pd.DataFrame(creation1,columns = ['city#']) # add them to the dataframe
df3 = pd.DataFrame(creation2, columns = ['phone#'])
df = pd.concat([df,df2,df3],axis =1 )

In [ ]:
len(df)

In [ ]:
df = df.drop(['city','phone'], axis=1) # no need for old columns

In [ ]:

#final cleaning step - change true/false labels to 1/0's. 
UB = []
for i in range(len(df)):
    if df['ultimate_black_user'][i] == True:
        UB.append([1]) 
    else:
        UB.append([0])
df4 = pd.DataFrame(UB,columns = ['UB'])
df4.head()

In [ ]:
df = pd.concat([df,df4],axis =1 )
df = df.drop(['ultimate_black_user'], axis=1) # no need for old columns
df.head() #ready for model

In [ ]:
#splitting into features and target
y = df['active_users']
X = df[['avg_dist', 'avg_rating_by_driver','trips_in_first_30_days', 'city#', 'phone#','UB']]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=42)

# Create the classifier: logreg
logreg = LogisticRegression()

# Fit the classifier to the training data
logreg.fit(X_train,y_train)

# Predict the labels of the test set: y_pred
y_pred = logreg.predict(X_test)

# Compute and print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import roc_curve

# Compute predicted probabilities: y_pred_prob
y_pred_prob = logreg.predict_proba(X_test)[:,1]

# Generate ROC curve values: fpr, tpr, thresholds
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

# Compute predicted probabilities: y_pred_prob
y_pred_prob = logreg.predict_proba(X_test)[:,1]

# Compute and print AUC score
print("AUC: {}".format(roc_auc_score(y_test, y_pred_prob)))

# Compute cross-validated AUC scores: cv_auc
cv_auc = cross_val_score(logreg, X, y, cv = 5,scoring = 'roc_auc')

# Print list of AUC scores
print("AUC scores computed using 5-fold cross-validation: {}".format(cv_auc))

In [ ]:

# Setup the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space}
# Instantiate a logistic regression classifier: logreg
logreg = LogisticRegression()

# Instantiate the GridSearchCV object: logreg_cv
logreg_cv = GridSearchCV(logreg, param_grid, cv=5)

# Fit it to the data
logreg_cv.fit(X,y)

# Print the tuned parameters and score
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
# fit model to training data
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
feat_imp = pd.Series(model.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [ ]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

#### Conclusion

Active users are about 24%

Model prediction is 78%, 

The idea of the "sixth month" requires more clarity. 

The use of Logistic regression was used as predictor of binary classification. 

Average distance was the most important feature. Users who use the service for local, smaller trips are most likely to engage frequently and remain active. 

RECOMMENDATION

Focus on retaining local users. 
Improve Android App for users